In [6]:
import pandas as pd
import numpy as np
import os
import pickle
import re
from collections import Counter
from tqdm import tqdm

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AutoTokenizer

import ilm.ilm.tokenize_util
from ilm.ilm.infer import infill_with_ilm
from perturbation_functions import calculate_necc_and_suff, gen_num_samples_table, gen_probs_table

In [ ]:
# MODEL_DIR = './ilm/Models/ILM/'
# MASK_CLS = 'ilm.ilm.mask.hierarchical.MaskHierarchical'

# tokenizer = ilm.ilm.tokenize_util.Tokenizer.GPT2
# with open(os.path.join(MODEL_DIR, 'additional_ids_to_tokens.pkl'), 'rb') as f:
#     additional_ids_to_tokens = pickle.load(f)
# additional_tokens_to_ids = {v:k for k, v in additional_ids_to_tokens.items()}
# try:
#     ilm.ilm.tokenize_util.update_tokenizer(additional_ids_to_tokens, tokenizer)
# except ValueError:
#     print('Already updated')
# print(additional_tokens_to_ids)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = GPT2LMHeadModel.from_pretrained(MODEL_DIR)


In [7]:
MODEL_DIR = './ilm/Models/ILM/'

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = GPT2LMHeadModel.from_pretrained("openai-community/gpt2")

with open(os.path.join(MODEL_DIR, 'additional_ids_to_tokens.pkl'), 'rb') as f:
    additional_ids_to_tokens = pickle.load(f)
additional_tokens_to_ids = {v:k for k, v in additional_ids_to_tokens.items()}
try:
    ilm.ilm.tokenize_util.update_tokenizer(additional_ids_to_tokens, tokenizer)
except ValueError:
    print('Already updated')
print(additional_tokens_to_ids)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()
_ = model.to(device)


Already updated
{'<|startofinfill|>': 50257, '<|endofinfill|>': 50258, '<|infill_document|>': 50259, '<|infill_paragraph|>': 50260, '<|infill_sentence|>': 50261, '<|infill_ngram|>': 50262, '<|infill_word|>': 50263}


In [ ]:
test_suite_cases = pd.read_csv("hatecheck-data/test_suite_cases.csv", index_col="case_id") #.drop(columns=['Unnamed:0'])
test_suite_cases.drop(columns=['Unnamed: 0'], inplace=True)
# target_ds = ['women', 'Muslims']
# funcs = ['derog_neg_emote_h', 'derog_neg_attrib_h', 'derog_dehum_h']
# 
# test_suite_cases = test_suite_cases[test_suite_cases.target_ident.isin(target_ds) & 
                                    # test_suite_cases.functionality.isin(funcs)]
# tts = [text for _, text in test_suite_cases.test_case.items()]
# 
# with open("Data/HateCheck_test_suite_cases.txt", "w") as f:
    # f.write("\n".join(tts))

In [71]:
test_suite_cases["target_ident"].unique()

array(['women', 'trans people', 'gay people', 'black people',
       'disabled people', 'Muslims', 'immigrants', nan], dtype=object)

In [70]:
test_suite_cases.head(10)

,functionality,test_case,label_gold,target_ident,direction,focus_words,focus_lemma,ref_case_id,ref_templ_id,templ_id,case_templ
case_id,,,,,,,,,,,
1,derog_neg_emote_h,I hate women.,hateful,women,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
2,derog_neg_emote_h,I hate trans people.,hateful,trans people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
3,derog_neg_emote_h,I hate gay people.,hateful,gay people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
4,derog_neg_emote_h,I hate black people.,hateful,black people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
5,derog_neg_emote_h,I hate disabled people.,hateful,disabled people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
6,derog_neg_emote_h,I hate Muslims.,hateful,Muslims,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
7,derog_neg_emote_h,I hate immigrants.,hateful,immigrants,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
8,derog_neg_emote_h,I really can't stand women like you.,hateful,women,directed,can't stand,can't stand,NaN,NaN,2,I really can't stand [IDENTITY_P] like you.
9,derog_neg_emote_h,I really can't stand trans people like you.,hateful,trans people,directed,can't stand,can't stand,NaN,NaN,2,I really can't stand [IDENTITY_P] like you.


In [80]:
len(test_suite_cases[pd.notna(test_suite_cases['target_ident'])]['case_templ'].unique())


553

845

In [ ]:
def add_word_hatecheck(word:str, df:pd.DataFrame):
    indices = []
    for x in df.loc[df['target_ident']!='nan'].unique():
        index = int((df["case_templ"] == x).idxmax())
        indices.append(index)
    return indices

li = add_word_hatecheck('men', test_suite_cases)

In [62]:
print(li)

[1, 8, 15, 22, 29, 36, 43, 50, 57, 64, 71, 78, 85, 92, 99, 106, 113, 120, 127, 134, 141, 148, 155, 162, 169, 176, 183, 190, 197, 204, 211, 218, 225, 232, 239, 246, 253, 260, 267, 274, 281, 288, 295, 302, 309, 316, 323, 330, 337, 344, 351, 358, 365, 372, 379, 386, 393, 400, 407, 414, 421, 428, 435, 442, 449, 456, 463, 470, 477, 484, 491, 498, 505, 512, 519, 526, 533, 540, 547, 554, 561, 568, 575, 582, 589, 596, 603, 610, 617, 631, 638, 645, 652, 659, 666, 673, 680, 687, 694, 701, 708, 715, 722, 729, 736, 743, 750, 757, 764, 771, 778, 785, 792, 799, 806, 813, 820, 827, 834, 841, 859, 895, 913, 931, 949, 985, 1003, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 10

In [ ]:
# generate approximately 100 perturbations for each token. 
num_samples = gen_num_samples_table(20, 100)
probs_table = gen_probs_table(20)
mask_tokn = additional_tokens_to_ids['<|infill_ngram|>']

orig_texts = []
necc_perturbed = []
suff_perturbed = []
necc_masks = []
suff_masks = []

with open("Data/HateCheck_test_suite_cases.txt", "r") as ff:
    with tqdm(total=120) as pbar:
        for text in ff:
            necc_pp, suff_pp, necc_mm, suff_mm = calculate_necc_and_suff(text, ilm_tokenizer=tokenizer, ilm_model=model, cl_tokenizer=None, cl_model=None, num_samples=num_samples,
                               mask_tokn=mask_tokn, additional_tokens_to_ids=additional_tokens_to_ids, probs_table=probs_table, 
                               return_pert_only=True)

            orig_texts.append(text)
            necc_perturbed.append(necc_pp)
            suff_perturbed.append(suff_pp)
            necc_masks.append(necc_mm)
            suff_masks.append(suff_mm)
            pbar.update(1)
    
necc_suff_perturbations = {'orig_texts': orig_texts, 
                           'necc_perturbed': necc_perturbed, 
                           'suff_perturbed': suff_perturbed,
                           'necc_masks': necc_masks,
                           'suff_masks': suff_masks}

#pickle.dump(necc_suff_perturbations, open('Data/HateCheck_necc_suff_perturbations.pickle', 'wb'))
pickle.dump(necc_suff_perturbations, open('Data/HateCheck_necc_suff_perturbations_2.pickle', 'wb'))

In [ ]:
with open("Data/hatecheck_perturbations/orig_texts.txt", "w") as ff:
    ff.write("\n".join(necc_suff_perturbations['orig_texts']))

In [ ]:
with open("Data/hatecheck_perturbations/necc_perturbations.tsv", "w") as ff:
    for ll in necc_suff_perturbations['necc_perturbed']:
        ff.write("\t".join(ll))
        ff.write("\n")

In [ ]:
with open("Data/hatecheck_perturbations/suff_perturbations.tsv", "w") as ff:
    for ll in necc_suff_perturbations['suff_perturbed']:
        ff.write("\t".join(ll))
        ff.write("\n")

In [ ]:
with open("Data/hatecheck_perturbations/necc_masks.tsv", "w") as ff: 
    for ll in necc_suff_perturbations['necc_masks']:
        llist = ll.astype(int).astype(str).tolist()
        ff.write("\t".join([" ".join(ii) for ii in llist]))
        ff.write("\n")

In [ ]:
with open("Data/hatecheck_perturbations/suff_masks.tsv", "w") as ff: 
    for ll in necc_suff_perturbations['suff_masks']:
        llist = ll.astype(int).astype(str).tolist()
        ff.write("\t".join([" ".join(ii) for ii in llist]))
        ff.write("\n")